In [64]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

## Read the data

In [65]:
# rfr file, rm file
file_name_rfr = r"..\Sray\original dirty data\New_rfr.xlsx"
file_name_rm = r"..\Sray\original dirty data\New_rm.xlsx"

#get the sorted fin file
file_name_str = r"..\Sray\Analysis_bottom\AU\AU_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\CN\CN_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\HK\HK_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\MY\MY_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\SG\SG_Bloomberg_fin_data_portfolio.xlsx"



indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'historical_market_cap'
# indicator = 'roa'
# indicator = 'vol_180d'
## indicator = 'return_com_eqy'

file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)
rfr_file = pd.ExcelFile(file_name_rfr)
rm_file = pd.ExcelFile(file_name_rm)

findata_sheet = pd.read_excel(
    fin_file, sheet_name="main", index_col=0, header=0)

porfolio_selection = pd.DataFrame(findata_sheet)


porfolio_selection.head()


,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
64,BEACH ENERGY LTD,BPT AU Equity,1,2017,2017-12-31,63.715923,55.934765,56.559856,78.597229,3163.662269,...,-0.231913,26.615385,-46.902965,143.144459,0.513595,0.244650,-16.640570,25.681782,1.426979,31.980287
64,BEACH ENERGY LTD,BPT AU Equity,1,2018,2018-12-31,63.917110,52.823921,53.990326,84.858521,2851.980723,...,0.150097,162.555556,-42.855693,144.975789,1.416866,0.450774,-6.930570,31.560391,0.607143,32.183942
64,BEACH ENERGY LTD,BPT AU Equity,1,2019,2019-12-31,58.867317,38.447598,53.204353,84.858521,2045.579562,...,-0.335546,-33.146067,-65.040082,199.914059,0.159539,-0.114482,-9.693295,23.544837,0.995420,29.376417
64,BEACH ENERGY LTD,BPT AU Equity,1,2020,2020-12-31,58.867317,38.447598,53.204353,84.858521,2045.579562,...,-0.236037,-29.787234,-19.155449,78.119710,-0.136096,-0.159078,-14.615757,23.006446,0.995423,29.354120
64,BEACH ENERGY LTD,BPT AU Equity,1,2021,2021-12-31,56.443012,31.168831,53.204353,84.858521,1658.317491,...,-0.177347,110.472973,-36.657296,246.943174,0.595806,0.102629,-6.417029,22.538936,0.673037,28.272821


## Sorting the data by firm name and then date

In [66]:
portfolio_selection_sorted = porfolio_selection.sort_values(
    by=['firm_name', 'date'])
portfolio_selection_sorted.head(10)

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
346,29METALS LTD,29M AU Equity,2,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-2.478196,45.795635,NaN,NaN,-9.662817,37.771376,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2018,2018-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-16.507780,100.808472,NaN,NaN,-8.171819,41.974506,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2019,2019-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-27.039477,118.522743,NaN,NaN,-9.197116,38.048781,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2020,2020-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.587213,52.219266,NaN,NaN,-9.134804,29.156100,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2021,2021-12-31,NaN,NaN,NaN,NaN,NaN,...,0.468829,-8.493156,-21.447492,107.377879,0.120643,0.385934,4.190439,35.369566,0.147600,NaN
346,29METALS LTD,29M AU Equity,2,2022,2022-12-30,32.863895,5.708245,29.201935,63.576160,166.691793,...,-0.015473,-11.694764,-34.498056,147.401152,0.154702,0.459116,0.098781,38.126217,0.244247,16.661506
346,29METALS LTD,29M AU Equity,2,2023,2023-01-31,NaN,NaN,NaN,NaN,NaN,...,0.024564,-11.396429,-30.772953,125.052021,0.154948,0.501355,0.098781,38.126217,0.244247,NaN
75,A2B AUSTRALIA LTD,A2B AU Equity,3,2017,2017-12-31,26.798109,0.906071,12.575574,66.767006,11.394363,...,-0.219742,4.433679,10.298060,30.841917,-0.190143,0.475697,2.450075,10.992115,0.101175,13.636903
75,A2B AUSTRALIA LTD,A2B AU Equity,3,2018,2018-12-31,26.285082,0.000000,11.940750,66.767006,0.000000,...,-0.229590,3.598049,7.360237,29.933450,-0.125685,-0.321829,4.425666,6.625187,-0.043375,12.981626


## Ranking stocks percentile
| Country/Region | Percentile chosen (Top(%))|
| --- | --- |
| Australia (AU)| 20 |
| China (CN)| 20 |
| Malaysia (SG)| 20 |
| Hong Kong (SG)| 25 |
| Singapore (SG)| 25 |

## Excess Return
At the end of each month, compute the cross-sectional portfolio return of esg_bottom20. For example, at the end of August, after selecting the bottom 20% stocks based on August ESG scores, compute the raw returns of each stocks in the high-rated portfolio (the bottom 20%) by the following equation: $$ret_{i}  = P_{1}/P_{0} -1 $$(for example, after ranking stocks at the end of Aug, calculate the individual returns by using $$(Sept-price/Aug-price) – 1)$$ Next, compute excess returns of each stocks: $$r_{i} = ret_{i} – r_{f}$$ where r_i is the excess return of stock i and r_f is the respective country’s monthly risk-free rate. So now we have r_1, …, r_n where n = number of stocks in the high-rated portfolio. Next, we equal-weight (average) all the r in the esg_bottom 20 and find the portfolio mean (denoted as μ_t) for each month. This generates the High portfolio excess return (equal weight) in the Excel file for each month. Scroll down the Excel file, compute the mean of all the months’ excess returns, and test for significance (H0: mean > 0). Indicate the t-stat value.

In [67]:
"""
change data format
"""

portfolio_selection_sorted['date'] = portfolio_selection_sorted['date'].astype(
    'datetime64')
portfolio_selection_sorted.dtypes


firm_name                        object
tkr                              object
sec_no                            int64
year                              int64
date                     datetime64[ns]
esg                             float64
e                               float64
s                               float64
g                               float64
es                              float64
eg                              float64
sg                              float64
esg_cubic                       float64
px                              float64
fcf                             float64
pe                              float64
beta                            float64
roa                             float64
vol_180d                        float64
sales                           float64
ret                             float64
historical_market_cap           float64
ep                              float64
ep_sec_mean                     float64
ep_sec_sd                       float64


In [68]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
quantile_rank = 0
quantile_path = ''
quantile_folder = ''


quantile_folder = "..\Sray\Analysis_bottom\\" + file_name[:2] + "\\" + indicator
quantile_path = quantile_folder + "\\" + file_name[:2] +"_portfolio_" + indicator + "_quantile_rank.xlsx"
bottom_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_bottom20.xlsx"
reporting_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_low.xlsx"
rfr_sheet = pd.read_excel(rfr_file, sheet_name= file_name[:2] )
rm_sheet = pd.read_excel(rm_file, sheet_name= file_name[:2] )

try:
    os.makedirs(quantile_folder)
except FileExistsError:
    # directory already exists
    pass

if file_name[:2].lower() == 'AU'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'CN'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'HK'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'MY'.lower():
    quantile_rank = 4

elif file_name[:2].lower() == 'SG'.lower():
    quantile_rank = 4
    

In [69]:
rfr_sheet = rfr_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rfr_yr': 'rfr'}, axis=1)
rfr_sheet

,year,rfr
0,2017,0.018870
1,2018,0.000634
2,2019,0.010361
3,2020,0.000686
4,2021,0.000571


In [70]:
rm_sheet = rm_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rm_yr': 'rm'}, axis=1)
rm_sheet

,year,rm
0,2017,0.094229
1,2018,-0.111802
2,2019,0.096939
3,2020,-0.094601
4,2021,0.136392


In [71]:
rfr_rm_sheet = pd.merge(rfr_sheet, rm_sheet, how='left', on='year')
rfr_rm_sheet

,year,rfr,rm
0,2017,0.018870,0.094229
1,2018,0.000634,-0.111802
2,2019,0.010361,0.096939
3,2020,0.000686,-0.094601
4,2021,0.000571,0.136392


In [72]:
"""
gets all the companies, dropping the empty columns
"""
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates()

#set the data to a list
companies = companies.tolist()


#ensure the number of companies exist
len(companies)


334

In [73]:
# new_companies_data.to_excel("SG_data_2.28_esgj3mean.xlsx")

### Rank the companies 

In [74]:
companies_sray = []
all_quantile_rank = []

initial_year = 2017
stop_year = 2023


for year in range(initial_year, stop_year+1, 1):

        # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
        temp_dataframe = portfolio_selection_sorted.loc[(
            portfolio_selection_sorted['year'] == year)].copy()

        
        """
        sort by the relative value and then divide them to relative division
        """

        temp_dataframe.sort_values(by=[indicator], inplace=True)

        temp_dataframe['quantile_rank'] = pd.qcut(temp_dataframe[indicator],
                                                    q=quantile_rank, labels=False, duplicates='drop')

        temp_dataframe_quantile = temp_dataframe.copy()

        companies_sray.append(temp_dataframe)

        all_quantile_rank.append(temp_dataframe_quantile)


In [75]:
all_sort_companies_data = pd.concat(all_quantile_rank)

all_sort_companies_data.to_excel(
    quantile_path, sheet_name="main")


In [76]:
sort_companies_data = pd.concat(companies_sray)
sort_companies_data.head()

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
firm no,,,,,,,,,,,,,,,,,,,,,
316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,0.0,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,0.0,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,0.0,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,0.0,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,0.0,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


In [77]:
all_sort_companies_data = all_sort_companies_data.reset_index()
all_sort_companies_data = pd.merge(all_sort_companies_data, rfr_rm_sheet, how='left', on='year')
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('ret')+1, 'rfr', all_sort_companies_data.pop('rfr'))
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('rfr')+1, 'rm', all_sort_companies_data.pop('rm'))

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


In [78]:
df_sec_company_count_sum = []
df_sec_company_count = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].count())
df_sec_company_count = df_sec_company_count.rename({'tkr':'tkr_count'}, axis=1)
df_sec_company_nunique = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].nunique())
df_sec_company_nunique = df_sec_company_nunique.rename({'tkr':'tkr_nunique'}, axis=1)

df_sec_company_count = pd.merge(df_sec_company_count, df_sec_company_nunique, on=['quantile_rank','year'])

In [79]:
df_sec_company_count_sum = list()
# for rank in range (0, quantile_rank, 1):
groups = df_sec_company_count.groupby(level='quantile_rank')

    # loop through the groups to create separate dataframes
dfs = {}
for name, group in groups:
    dfs[name] = group.copy()
        
    # display the separate dataframes
for name, df in dfs.items():

        # Create a new row with the sum of all values above it
    sum_row = df.sum()
    sum_index = (name, 'Sum')
    sum_row.name = sum_index

        # Append the new row to the original DataFrame
    temp_1 = df.append(sum_row)
    df_sec_company_count_sum.append(temp_1)
# df_sec_company_count_all = pd.concat(df_sec_company_count_sum)
df_sec_company_count_sum_pd = pd.concat(df_sec_company_count_sum)

In [80]:
book = load_workbook(quantile_path)
writer = pd.ExcelWriter(quantile_path, engine = 'openpyxl')
writer.book = book
df_sec_company_count_sum_pd.to_excel(writer, sheet_name='quantile sec no', index=True)
writer.close()

In [81]:
# sort_companies_data['date'] = sort_companies_data['date'].dt.strftime(
#     '%Y-%m')

all_sort_companies_data = all_sort_companies_data.sort_values(
    by=["date", "quantile_rank"])

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


**SRAY- Ret already calculated using excel by Dr May**
<br>**BLOOMBERG- Ret calculated using PYTHON CODE 2_portfolio.ipynb**

In [82]:

sort_companies_data_bottom20per = all_sort_companies_data.query("quantile_rank == 0")

sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


In [83]:
sort_companies_data_bottom20per.columns

Index(['firm no', 'firm_name', 'tkr', 'sec_no', 'year', 'date', 'esg', 'e',
       's', 'g', 'es', 'eg', 'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta',
       'roa', 'vol_180d', 'sales', 'ret', 'rfr', 'rm', 'historical_market_cap',
       'ep', 'ep_sec_mean', 'ep_sec_sd', 'ep_z', 'sp', 'sp_sec_mean',
       'sp_sec_sd', 'sp_z', 'cfp', 'cfp_sec_mean', 'cfp_sec_sd', 'cfp_z',
       'value', 'roa_sec_mean', 'roa_sec_sd', 'roa_z', 'esg_value',
       'quantile_rank'],
      dtype='object')

In [84]:
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates().tolist()
print(companies)

['29METALS LTD', 'A2B AUSTRALIA LTD', 'ABACUS PROPERTY GROUP', 'ACCENT GROUP LTD', 'ACCENT RESOURCES NL', 'ACRUX LTD', 'ADAIRS LTD', 'AGL ENERGY LTD', 'AIC MINES LTD', 'AINSWORTH GAME TECHNOLOGY LT', 'AJ LUCAS GROUP LTD', 'ALEXIUM INTERNATIONAL GROUP', 'ALKANE RESOURCES LTD', 'ALLKEM LTD', 'ALS LTD', 'ALTERITY THERAPEUTICS LTD', 'ALTIUM LTD', 'ALUMINA LTD', 'AMA GROUP LTD', 'ANSELL LTD', 'ANZ GROUP HOLDINGS LTD', 'APA GROUP', 'APM HUMAN SERVICES INTERNATI', 'ARA US HOSPITALITY TRUST', 'ARAFURA RARE EARTHS LTD', 'ARB CORP LTD', 'ARDENT LEISURE GROUP LTD', 'ARENA REIT', 'ARGENT MINERALS LTD', 'ARISTOCRAT LEISURE LTD', 'ASPEN GROUP', 'ASX LTD', 'ATLAS ARTERIA', 'AUB GROUP LTD', 'AUDINATE GROUP LTD', 'AURELIA METALS LTD', 'AURIZON HOLDINGS LTD', 'AUSSIE BROADBAND LTD', 'AUSTAL LTD', 'AUSTIN ENGINEERING LTD', 'AUSTRALIAN AGRICULTURAL CO', 'AUSTRALIAN ETHICAL INVT LTD', 'AUSTRALIAN FINANCE GROUP LTD', 'AUSTRALIAN VINTAGE LTD', 'AVECHO BIOTECHNOLOGY LTD', 'BABY BUNTING GROUP LTD', 'BANK OF QU

In [85]:
companies_sec=[]
initial_year=2017
stop_year=2023

company_dict1 = {}
company_dict_all = {}
sum_esg_sray = {}
sum_ret_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =0 and save it into company dictionary 
for year in range(initial_year,stop_year+1,1):    

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        each_year_companies_all_quantile = all_sort_companies_data.loc[(
            all_sort_companies_data['year'] == year)]

        sum = each_year_companies[indicator].sum()
        sum_esg_sray.setdefault(year, sum)

        sum_ret = each_year_companies['ret'].sum()
        sum_ret_dict.setdefault(year, sum_ret)

        companies = each_year_companies.iloc[:, 0]
        companies= companies.tolist() 

        total_companies = (each_year_companies_all_quantile.iloc[:, 0]).tolist()


        for c in range (len(companies)):
            company_dict1.setdefault(year,[]).append(companies[c])
            #company_dict2.setdefault(start_date,[]).append(companies[c])
        for c_all in range(len(total_companies)):
            company_dict_all.setdefault(
                year, []).append(total_companies[c_all])    
        

#make sure the list is correct     
#print(company_dict)
# print(f"company_dict1: {company_dict1}")
print(len(company_dict1))
print(f"sum_esg_sray_dic: {sum_esg_sray}")
print(f"sum_ret_dict: {sum_ret_dict}")
# print(len(company_dict2))


6
sum_esg_sray_dic: {2017: 38.32679596543312, 2018: 81.24434530735016, 2019: 7.882818132638932, 2020: 20.688620150089264, 2021: 2.657808065414432, 2022: 7.792210161685945, 2023: 0.0}
sum_ret_dict: {2017: 0.0, 2018: -13.670590300036448, 2019: 32.672510264075015, 2020: 112.18335458613912, 2021: 14.266032352226212, 2022: -17.308419262928734, 2023: 0.0}


If it is equal-weighted, compute the mean of the stocks returns in the portfolio. 
$μ_(p,t)=  (\sum ret_{(i,t)} )/n$; i = 1, 2, 3, …, n
Where $ret_(i,t) = return-of-stock-i-at-time-t$, n = number of stocks in the portfolio


In [86]:
no_of_companies = {}
equal_weight_dict = {}
for date_range, company_names in company_dict1.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    #equal weightage
    # equal_weight = np.repeat(1/len(v), len(v))
    equal_weight = 1/len(company_names)
        # print(f"factor_weight: {factor_weight}")

    equal_weight_dict.setdefault(date_range, float(equal_weight))
    no_of_companies.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"equal_weight: {equal_weight_dict}")
print(f"no_of_companies: {no_of_companies}")


equal_weight: {2017: 0.006060606060606061, 2018: 0.005780346820809248, 2019: 0.007692307692307693, 2020: 0.007874015748031496, 2021: 0.0136986301369863, 2022: 0.014084507042253521}
no_of_companies: {2017: 165.0, 2018: 173.0, 2019: 130.0, 2020: 127.0, 2021: 73.0, 2022: 71.0}


### total companies from all quantile

In [87]:
no_of_companies_all = {}
for date_range, company_names in company_dict_all.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    no_of_companies_all.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"no_of_companies: {no_of_companies_all}")


no_of_companies: {2017: 334.0, 2018: 334.0, 2019: 334.0, 2020: 334.0, 2021: 334.0, 2022: 334.0, 2023: 334.0}


## Calculate the number of companies (Bottom 20%) based on the date

In [88]:
no_of_companies_items = no_of_companies.items()
no_of_companies_list = list(no_of_companies_items)

no_of_companies_df = pd.DataFrame(no_of_companies_list)
no_of_companies_df.columns = ['year', 'no of firms(bottom 20)']
no_of_companies_df.head()

,year,no of firms(bottom 20)
0,2017,165.0
1,2018,173.0
2,2019,130.0
3,2020,127.0
4,2021,73.0


## Calculate the number of companies based on the date

In [89]:
no_of_companies_all_items = no_of_companies_all.items()
no_of_companies_all_list = list(no_of_companies_all_items)

no_of_companies_all_df = pd.DataFrame(no_of_companies_all_list)
no_of_companies_all_df.columns = ['year', 'no of firms']
no_of_companies_all_df.head()

,year,no of firms
0,2017,334.0
1,2018,334.0
2,2019,334.0
3,2020,334.0
4,2021,334.0


### Make the equal weight item into dataframe
Purpose: use for merging later on

In [90]:
equal_weight_items = equal_weight_dict.items()
equal_weight_list = list(equal_weight_items)

equal_weight_df = pd.DataFrame(equal_weight_list)
equal_weight_df.columns = ['year', 'eq_wg']
equal_weight_df.head()


,year,eq_wg
0,2017,0.006061
1,2018,0.005780
2,2019,0.007692
3,2020,0.007874
4,2021,0.013699


In [91]:
# no_of_companies_df.to_excel("eq_wg_no_of_companies_bottom20.xlsx")

In [92]:
sum_ret_items = sum_ret_dict.items()
sum_ret_list = list(sum_ret_items)

sum_ret_df = pd.DataFrame(sum_ret_list)
sum_ret_df.columns = ['year', 'sum_ret']
sum_ret_df.replace(0, np.nan, inplace=True)
sum_ret_df.head()


,year,sum_ret
0,2017,NaN
1,2018,-13.670590
2,2019,32.672510
3,2020,112.183355
4,2021,14.266032


### Make the sum of the esg_sray into dataframe
Purpose: merge into complete dataframe to make the data to be able to vefiry

In [93]:
sum_esg_sray_dic_items = sum_esg_sray.items()
sum_esg_sray_list = list(sum_esg_sray_dic_items)

sum_esg_sray_df = pd.DataFrame(sum_esg_sray_list)
sum_esg_sray_df.columns = ['year', 'sum_'+indicator]
sum_esg_sray_df.head()


,year,sum_e
0,2017,38.326796
1,2018,81.244345
2,2019,7.882818
3,2020,20.688620
4,2021,2.657808


### Merging with the data of top 20% **only**

In [94]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    equal_weight_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    sum_esg_sray_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    sum_ret_df, on='year')
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_e,sum_ret
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0,0.006061,38.326796,NaN
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0,0.006061,38.326796,NaN
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0,0.006061,38.326796,NaN
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0,0.006061,38.326796,NaN
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0,0.006061,38.326796,NaN


### Calculate the factor weight

In [95]:
if sort_companies_data_bottom20per[indicator].empty and sort_companies_data_bottom20per['sum_'+indicator].empty:
    sort_companies_data_bottom20per['fc_wg'] = None
else:
    sort_companies_data_bottom20per['fc_wg'] = sort_companies_data_bottom20per[indicator] / \
        sort_companies_data_bottom20per['sum_'+indicator]

sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_e,sum_ret,fc_wg
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0,0.006061,38.326796,NaN,0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0,0.006061,38.326796,NaN,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0,0.006061,38.326796,NaN,0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0,0.006061,38.326796,NaN,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0,0.006061,38.326796,NaN,0.0


In [96]:
sort_companies_data_bottom20per['r'] = (
    sort_companies_data_bottom20per['ret'] - sort_companies_data_bottom20per['rfr'])

"""
r = ret -rfr
"""
sort_companies_data_bottom20per['eq_wg_r'] = (
    sort_companies_data_bottom20per['r'] * sort_companies_data_bottom20per['eq_wg'])
sort_companies_data_bottom20per['fc_wg_r'] = (
    sort_companies_data_bottom20per['r'] * sort_companies_data_bottom20per['fc_wg'])

"""
ret
"""
sort_companies_data_bottom20per['eq_wg_ret'] = (
    sort_companies_data_bottom20per['ret'] * sort_companies_data_bottom20per['eq_wg'])
sort_companies_data_bottom20per['fc_wg_ret'] = (
    sort_companies_data_bottom20per['ret'] * sort_companies_data_bottom20per['fc_wg'])

sort_companies_data_bottom20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,quantile_rank,eq_wg,sum_e,sum_ret,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
5,298,LOVISA HOLDINGS LTD,LOV AU Equity,4,2017,2017-12-31,24.424103,0.0,9.552600,63.576160,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
6,174,LYNAS RARE EARTHS LTD,LYC AU Equity,2,2017,2017-12-31,27.844280,0.0,22.339783,61.077663,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
7,233,MAXIPARTS LTD,MXI AU Equity,3,2017,2017-12-31,23.387989,0.0,5.199516,64.810356,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
8,91,MCPHERSON'S LTD,MCP AU Equity,5,2017,2017-12-31,28.880394,0.0,10.368803,76.098740,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN
9,134,INSIGNIA FINANCIAL LTD,IFL AU Equity,7,2017,2017-12-31,27.713511,0.0,6.862152,76.098740,...,0.0,0.006061,38.326796,NaN,0.0,NaN,NaN,NaN,NaN,NaN


## Excess return portfolio
### Equal Weight and Factor Weight

In [97]:
initial_year = 2017
stop_year = 2023

lo_eq_wg_r_p_dict = {}
lo_fc_wg_r_p_dict = {}
eq_wg_ret_p_dict = {}
fc_wg_ret_p_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        """
        r
        """
        lo_eq_wg_r_p = each_year_companies['eq_wg_r'].sum()
        lo_fc_wg_r_p = each_year_companies['fc_wg_r'].sum()


        """
        ret
        """
        eq_wg_ret_p = each_year_companies['eq_wg_ret'].sum()
        fc_wg_ret_p = each_year_companies['fc_wg_ret'].sum()


        lo_eq_wg_r_p_dict.setdefault(year, lo_eq_wg_r_p)
        lo_fc_wg_r_p_dict.setdefault(year, lo_fc_wg_r_p)

        eq_wg_ret_p_dict.setdefault(year, eq_wg_ret_p)
        fc_wg_ret_p_dict.setdefault(year, fc_wg_ret_p)


print(f"lo_eq_wg_r_p_dict: {lo_eq_wg_r_p_dict}")
print(f"lo_fc_wg_r_p_dict: {lo_fc_wg_r_p_dict}")
print(f"eq_wg_ret_p_dict: {eq_wg_ret_p_dict}")
print(f"fc_wg_ret_p_dict: {fc_wg_ret_p_dict}")
# print(len(company_dict2))


lo_eq_wg_r_p_dict: {2017: 0.0, 2018: -0.07964742369963265, 2019: 0.24120510203134635, 2020: 0.8826583038278674, 2021: 0.1948619226332358, 2022: 0.0, 2023: 0.0}
lo_fc_wg_r_p_dict: {2017: 0.0, 2018: -0.09670745632730816, 2019: 0.2685828539605652, 2020: 0.3871977341793557, 2021: 0.20382000140577683, 2022: 0.0, 2023: 0.0}
eq_wg_ret_p_dict: {2017: 0.0, 2018: -0.07902075317940144, 2019: 0.25132700203134634, 2020: 0.8833335006782608, 2021: 0.19542510071542754, 2022: -0.24378055299899615, 2023: 0.0}
fc_wg_ret_p_dict: {2017: 0.0, 2018: -0.09607345632730817, 2019: 0.27894385396056515, 2020: 0.3878837341793556, 2021: 0.20439100140577682, 2022: -0.2304570992114271, 2023: 0.0}


In [98]:
lo_eq_wg_r_p_dict_items = lo_eq_wg_r_p_dict.items()
lo_eq_wg_r_p_dict_list = list(lo_eq_wg_r_p_dict_items)

lo_eq_wg_r_p_dict_df = pd.DataFrame(lo_eq_wg_r_p_dict_list)
lo_eq_wg_r_p_dict_df.columns = ['year', 'lo_eq_wg_r_p']
lo_eq_wg_r_p_dict_df.replace(0, np.nan, inplace = True)
lo_eq_wg_r_p_dict_df.head()

,year,lo_eq_wg_r_p
0,2017,NaN
1,2018,-0.079647
2,2019,0.241205
3,2020,0.882658
4,2021,0.194862


In [99]:
lo_fc_wg_r_p_dict_items = lo_fc_wg_r_p_dict.items()
lo_fc_wg_r_p_dict_list = list(lo_fc_wg_r_p_dict_items)

lo_fc_wg_r_p_dict_df = pd.DataFrame(lo_fc_wg_r_p_dict_list)
lo_fc_wg_r_p_dict_df.columns = ['year', 'lo_fc_wg_r_p']
lo_fc_wg_r_p_dict_df.replace(0, np.nan, inplace=True)
lo_fc_wg_r_p_dict_df.head()


,year,lo_fc_wg_r_p
0,2017,NaN
1,2018,-0.096707
2,2019,0.268583
3,2020,0.387198
4,2021,0.203820


In [100]:
eq_wg_ret_p_dict_items = eq_wg_ret_p_dict.items()
eq_wg_ret_p_dict_list = list(eq_wg_ret_p_dict_items)

eq_wg_ret_p_dict_df = pd.DataFrame(eq_wg_ret_p_dict_list)
eq_wg_ret_p_dict_df.columns = ['year', 'eq_wg_ret_p']
eq_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_ret_p_dict_df.head()


,year,eq_wg_ret_p
0,2017,NaN
1,2018,-0.079021
2,2019,0.251327
3,2020,0.883334
4,2021,0.195425


In [101]:
fc_wg_ret_p_dict_items = fc_wg_ret_p_dict.items()
fc_wg_ret_p_dict_list = list(fc_wg_ret_p_dict_items)

fc_wg_ret_p_dict_df = pd.DataFrame(fc_wg_ret_p_dict_list)
fc_wg_ret_p_dict_df.columns = ['year', 'fc_wg_ret_p']
fc_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_ret_p_dict_df.head()


,year,fc_wg_ret_p
0,2017,NaN
1,2018,-0.096073
2,2019,0.278944
3,2020,0.387884
4,2021,0.204391


In [102]:
SG_esg_int = pd.merge(no_of_companies_all_df, no_of_companies_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_eq_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_fc_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,2017,334.0,165.0,NaN,NaN,NaN,NaN
1,2018,334.0,173.0,-0.079647,-0.096707,-0.079021,-0.096073
2,2019,334.0,130.0,0.241205,0.268583,0.251327,0.278944
3,2020,334.0,127.0,0.882658,0.387198,0.883334,0.387884
4,2021,334.0,73.0,0.194862,0.203820,0.195425,0.204391


### Merging the eq and fc wg of the r_p into the big dataframe

In [103]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_eq_wg_r_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_fc_wg_r_p_dict_df, on='year')

sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    eq_wg_ret_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    fc_wg_ret_p_dict_df, on='year')

sort_companies_data_bottom20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret,lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,298,LOVISA HOLDINGS LTD,LOV AU Equity,4,2017,2017-12-31,24.424103,0.0,9.552600,63.576160,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,174,LYNAS RARE EARTHS LTD,LYC AU Equity,2,2017,2017-12-31,27.844280,0.0,22.339783,61.077663,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,233,MAXIPARTS LTD,MXI AU Equity,3,2017,2017-12-31,23.387989,0.0,5.199516,64.810356,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,91,MCPHERSON'S LTD,MCP AU Equity,5,2017,2017-12-31,28.880394,0.0,10.368803,76.098740,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,134,INSIGNIA FINANCIAL LTD,IFL AU Equity,7,2017,2017-12-31,27.713511,0.0,6.862152,76.098740,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
sort_companies_data_bottom20per['rm_r_y1'] = (
    sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr']) * 12
sort_companies_data_bottom20per['rm_r_y2'] = (((1+(
    sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr'])) ** 12) - 1)

sort_companies_data_bottom20per['lo_eq_wg_r_p_y1'] = sort_companies_data_bottom20per['lo_eq_wg_r_p'] *12
sort_companies_data_bottom20per['lo_eq_wg_r_p_y2'] = (((1 + sort_companies_data_bottom20per['lo_eq_wg_r_p'])**12) - 1)
sort_companies_data_bottom20per['eq_r_alpha_y1'] = sort_companies_data_bottom20per['lo_eq_wg_r_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['eq_r_alpha_y2'] = sort_companies_data_bottom20per['lo_eq_wg_r_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,NaN,NaN,NaN,NaN,0.904308,1.391342,NaN,NaN,NaN,NaN
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,NaN,NaN,NaN,NaN,0.904308,1.391342,NaN,NaN,NaN,NaN
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,NaN,NaN,NaN,NaN,0.904308,1.391342,NaN,NaN,NaN,NaN
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,NaN,NaN,NaN,NaN,0.904308,1.391342,NaN,NaN,NaN,NaN
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,NaN,NaN,NaN,NaN,0.904308,1.391342,NaN,NaN,NaN,NaN


In [105]:
sort_companies_data_bottom20per['lo_fc_wg_r_p_y1'] = sort_companies_data_bottom20per['lo_fc_wg_r_p'] * 12
sort_companies_data_bottom20per['lo_fc_wg_r_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_fc_wg_r_p'])**12) - 1)
sort_companies_data_bottom20per['fc_r_alpha_y1'] = sort_companies_data_bottom20per['lo_fc_wg_r_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['fc_r_alpha_y2'] = sort_companies_data_bottom20per['lo_fc_wg_r_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:

# sort_companies_data_bottom20per.to_excel(
#     "../3.04/SG_esg_bottom20.xlsx", sheet_name="main")

## CAPM-adjusted return ($r_{c}$)

$r_(c,t)=r_(f,t)+β_(p,t) (r_(m,t)-r_(f,t))$

In [107]:
sort_companies_data_bottom20per['rc'] = sort_companies_data_bottom20per['rfr'] + \
    (sort_companies_data_bottom20per['beta'] *
    (sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr']))
    
"""
equal weight rc
"""
sort_companies_data_bottom20per['lo_eq_wg_rc_int'] = sort_companies_data_bottom20per ['eq_wg'] * sort_companies_data_bottom20per['rc']

"""
factor weight rc
"""
sort_companies_data_bottom20per['lo_fc_wg_rc_int'] = sort_companies_data_bottom20per['fc_wg'] * \
    sort_companies_data_bottom20per['rc']

sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.019940,-0.000121,-0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025110,0.000152,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.971664,-0.005889,-0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.134365,0.000814,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044959,0.000272,0.0


In [108]:
initial_year = 2017
stop_year = 2023

lo_eq_wg_rc_p_dict = {}
lo_fc_wg_rc_p_dict = {}

# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        """
        rc
        """
        lo_eq_wg_rc_p = each_year_companies['lo_eq_wg_rc_int'].sum()
        lo_fc_wg_rc_p = each_year_companies['lo_fc_wg_rc_int'].sum()

        lo_eq_wg_rc_p_dict.setdefault(year, lo_eq_wg_rc_p)
        lo_fc_wg_rc_p_dict.setdefault(year, lo_fc_wg_rc_p)


print(f"lo_eq_wg_rc_p_dict: {lo_eq_wg_rc_p_dict}")
print(f"lo_fc_wg_rc_p_dict: {lo_fc_wg_rc_p_dict}")


lo_eq_wg_rc_p_dict: {2017: 0.07040788635577457, 2018: -0.09805428538712194, 2019: 0.08816072345662732, 2020: -0.12472848971597802, 2021: 0.192390022818724, 2022: 0.0, 2023: 0.0}
lo_fc_wg_rc_p_dict: {2017: 0.0685928803930992, 2018: -0.08804349806872853, 2019: 0.051051936702048696, 2020: -0.13376166427833636, 2021: 0.22983000292714617, 2022: 0.0, 2023: 0.0}


In [109]:
lo_eq_wg_rc_p_dict_items = lo_eq_wg_rc_p_dict.items()
lo_eq_wg_rc_p_dict_list = list(lo_eq_wg_rc_p_dict_items)

lo_eq_wg_rc_p_dict_df = pd.DataFrame(lo_eq_wg_rc_p_dict_list)
lo_eq_wg_rc_p_dict_df.columns = ['year', 'lo_eq_wg_rc_p']
lo_eq_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
lo_eq_wg_rc_p_dict_df.head()


,year,lo_eq_wg_rc_p
0,2017,0.070408
1,2018,-0.098054
2,2019,0.088161
3,2020,-0.124728
4,2021,0.192390


In [110]:
lo_fc_wg_rc_p_dict_items = lo_fc_wg_rc_p_dict.items()
lo_fc_wg_rc_p_dict_list = list(lo_fc_wg_rc_p_dict_items)

lo_fc_wg_rc_p_dict_df = pd.DataFrame(lo_fc_wg_rc_p_dict_list).dropna()
lo_fc_wg_rc_p_dict_df.columns = ['year', 'lo_fc_wg_rc_p']
lo_fc_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
lo_fc_wg_rc_p_dict_df.head()


,year,lo_fc_wg_rc_p
0,2017,0.068593
1,2018,-0.088043
2,2019,0.051052
3,2020,-0.133762
4,2021,0.229830


In [111]:
SG_esg_int = pd.merge(SG_esg_int, lo_eq_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_fc_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()

,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_rc_p,lo_fc_wg_rc_p
0,2017,334.0,165.0,NaN,NaN,NaN,NaN,0.070408,0.068593
1,2018,334.0,173.0,-0.079647,-0.096707,-0.079021,-0.096073,-0.098054,-0.088043
2,2019,334.0,130.0,0.241205,0.268583,0.251327,0.278944,0.088161,0.051052
3,2020,334.0,127.0,0.882658,0.387198,0.883334,0.387884,-0.124728,-0.133762
4,2021,334.0,73.0,0.194862,0.203820,0.195425,0.204391,0.192390,0.229830


In [112]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_eq_wg_rc_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_fc_wg_rc_p_dict_df, on='year')

sort_companies_data_bottom20per.head(15)

,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int,lo_eq_wg_rc_p,lo_fc_wg_rc_p
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,NaN,NaN,NaN,NaN,NaN,-0.019940,-0.000121,-0.0,0.070408,0.068593
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,NaN,NaN,NaN,NaN,NaN,0.025110,0.000152,0.0,0.070408,0.068593
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,NaN,NaN,NaN,NaN,NaN,-0.971664,-0.005889,-0.0,0.070408,0.068593
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,NaN,NaN,NaN,NaN,NaN,0.134365,0.000814,0.0,0.070408,0.068593
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,NaN,NaN,NaN,NaN,NaN,0.044959,0.000272,0.0,0.070408,0.068593
5,298,LOVISA HOLDINGS LTD,LOV AU Equity,4,2017,2017-12-31,24.424103,0.0,9.552600,63.576160,...,NaN,NaN,NaN,NaN,NaN,0.011311,0.000069,0.0,0.070408,0.068593
6,174,LYNAS RARE EARTHS LTD,LYC AU Equity,2,2017,2017-12-31,27.844280,0.0,22.339783,61.077663,...,NaN,NaN,NaN,NaN,NaN,0.182640,0.001107,0.0,0.070408,0.068593
7,233,MAXIPARTS LTD,MXI AU Equity,3,2017,2017-12-31,23.387989,0.0,5.199516,64.810356,...,NaN,NaN,NaN,NaN,NaN,-0.084327,-0.000511,-0.0,0.070408,0.068593
8,91,MCPHERSON'S LTD,MCP AU Equity,5,2017,2017-12-31,28.880394,0.0,10.368803,76.098740,...,NaN,NaN,NaN,NaN,NaN,0.071538,0.000434,0.0,0.070408,0.068593
9,134,INSIGNIA FINANCIAL LTD,IFL AU Equity,7,2017,2017-12-31,27.713511,0.0,6.862152,76.098740,...,NaN,NaN,NaN,NaN,NaN,0.078517,0.000476,0.0,0.070408,0.068593


In [113]:
sort_companies_data_bottom20per['lo_eq_wg_rc_p_y1'] = sort_companies_data_bottom20per['lo_eq_wg_rc_p'] * 12
sort_companies_data_bottom20per['eq_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_eq_wg_rc_p'])**12) - 1)
sort_companies_data_bottom20per['eq_rc_alpha_y1'] = sort_companies_data_bottom20per['lo_eq_wg_rc_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['eq_rc_alpha_y2'] = sort_companies_data_bottom20per['eq_wg_rc_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int,lo_eq_wg_rc_p,lo_fc_wg_rc_p,lo_eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,NaN,-0.019940,-0.000121,-0.0,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,NaN,0.025110,0.000152,0.0,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,NaN,-0.971664,-0.005889,-0.0,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,NaN,0.134365,0.000814,0.0,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,NaN,0.044959,0.000272,0.0,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826


In [114]:
sort_companies_data_bottom20per['lo_fc_wg_rc_p_y1'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p'] * 12
sort_companies_data_bottom20per['lo_fc_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_fc_wg_rc_p'])**12) - 1)
sort_companies_data_bottom20per['fc_rc_alpha_y1'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['fc_rc_alpha_y2'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,lo_eq_wg_rc_p,lo_fc_wg_rc_p,lo_eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2,lo_fc_wg_rc_p_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y1,fc_rc_alpha_y2
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826,0.823115,1.216906,-0.081193,-0.174436
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826,0.823115,1.216906,-0.081193,-0.174436
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826,0.823115,1.216906,-0.081193,-0.174436
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826,0.823115,1.216906,-0.081193,-0.174436
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,0.070408,0.068593,0.844895,1.262516,-0.059413,-0.128826,0.823115,1.216906,-0.081193,-0.174436


In [115]:
SG_int_wanted = sort_companies_data_bottom20per[[
    'year', 'rm_r_y1', 'rm_r_y2', 'lo_eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_r_alpha_y2', 'lo_eq_wg_r_p_y2', 'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_r_alpha_y2', 'lo_fc_wg_r_p_y2', 'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_rc_alpha_y2', 'eq_wg_rc_p_y2', 'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_rc_alpha_y2', 'lo_fc_wg_rc_p_y2']]
SG_int_wanted = SG_int_wanted.drop_duplicates()
SG_int_wanted.head()


,year,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,eq_r_alpha_y1,eq_r_alpha_y2,lo_eq_wg_r_p_y2,lo_fc_wg_r_p_y1,fc_r_alpha_y1,fc_r_alpha_y2,lo_fc_wg_r_p_y2,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,lo_fc_wg_rc_p_y2
0,2017,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.844895,-0.059413,-0.128826,1.262516,0.823115,-0.081193,-0.174436,1.216906
165,2018,-1.349236,-0.761002,-0.955769,0.393467,0.130363,-0.630639,-1.160489,0.188746,0.056083,-0.704919,-1.176651,0.172584,0.050846,-0.710156,-1.056522,0.292714,0.091896,-0.669106
338,2019,1.038930,1.708499,2.894461,1.855531,10.661231,12.369730,3.222994,2.184064,14.662550,16.371049,1.057929,0.018998,0.047738,1.756237,0.612623,-0.426307,-0.890933,0.817566
468,2020,-1.143442,-0.699302,10.591900,11.735342,1982.404066,1981.704764,4.646373,5.789815,50.475531,49.776230,-1.496742,-0.353300,-0.098530,-0.797831,-1.605140,-0.461698,-0.122194,-0.821496
595,2021,1.629852,3.610187,2.338343,0.708491,3.858433,7.468620,2.445840,0.815988,4.652535,8.262723,2.308680,0.678829,3.650573,7.260760,2.757960,1.128108,7.361104,10.971292


In [116]:
SG_esg_int = pd.merge(SG_esg_int, SG_int_wanted,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_rc_p,lo_fc_wg_rc_p,rm_r_y1,...,fc_r_alpha_y2,lo_fc_wg_r_p_y2,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,lo_fc_wg_rc_p_y2
0,2017,334.0,165.0,NaN,NaN,NaN,NaN,0.070408,0.068593,0.904308,...,NaN,NaN,0.844895,-0.059413,-0.128826,1.262516,0.823115,-0.081193,-0.174436,1.216906
1,2018,334.0,173.0,-0.079647,-0.096707,-0.079021,-0.096073,-0.098054,-0.088043,-1.349236,...,0.056083,-0.704919,-1.176651,0.172584,0.050846,-0.710156,-1.056522,0.292714,0.091896,-0.669106
2,2019,334.0,130.0,0.241205,0.268583,0.251327,0.278944,0.088161,0.051052,1.038930,...,14.662550,16.371049,1.057929,0.018998,0.047738,1.756237,0.612623,-0.426307,-0.890933,0.817566
3,2020,334.0,127.0,0.882658,0.387198,0.883334,0.387884,-0.124728,-0.133762,-1.143442,...,50.475531,49.776230,-1.496742,-0.353300,-0.098530,-0.797831,-1.605140,-0.461698,-0.122194,-0.821496
4,2021,334.0,73.0,0.194862,0.203820,0.195425,0.204391,0.192390,0.229830,1.629852,...,4.652535,8.262723,2.308680,0.678829,3.650573,7.260760,2.757960,1.128108,7.361104,10.971292


In [117]:
SG_esg_int = SG_esg_int[['year', 'no of firms', 'no of firms(bottom 20)',
                            'rm_r_y1', 'rm_r_y2', 'eq_wg_ret_p', 'fc_wg_ret_p', 'lo_eq_wg_r_p',
                            'lo_eq_wg_r_p_y1', 'eq_r_alpha_y1', 'lo_eq_wg_r_p_y2', 'eq_r_alpha_y2', 'lo_fc_wg_r_p',
                            'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'lo_fc_wg_r_p_y2', 'fc_r_alpha_y2', 'lo_eq_wg_rc_p',
                            'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2', 'eq_rc_alpha_y2', 'lo_fc_wg_rc_p',
                            'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'lo_fc_wg_rc_p_y2', 'fc_rc_alpha_y2']]
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,2017,334.0,165.0,0.904308,1.391342,NaN,NaN,NaN,NaN,NaN,...,0.070408,0.844895,-0.059413,1.262516,-0.128826,0.068593,0.823115,-0.081193,1.216906,-0.174436
1,2018,334.0,173.0,-1.349236,-0.761002,-0.079021,-0.096073,-0.079647,-0.955769,0.393467,...,-0.098054,-1.176651,0.172584,-0.710156,0.050846,-0.088043,-1.056522,0.292714,-0.669106,0.091896
2,2019,334.0,130.0,1.038930,1.708499,0.251327,0.278944,0.241205,2.894461,1.855531,...,0.088161,1.057929,0.018998,1.756237,0.047738,0.051052,0.612623,-0.426307,0.817566,-0.890933
3,2020,334.0,127.0,-1.143442,-0.699302,0.883334,0.387884,0.882658,10.591900,11.735342,...,-0.124728,-1.496742,-0.353300,-0.797831,-0.098530,-0.133762,-1.605140,-0.461698,-0.821496,-0.122194
4,2021,334.0,73.0,1.629852,3.610187,0.195425,0.204391,0.194862,2.338343,0.708491,...,0.192390,2.308680,0.678829,7.260760,3.650573,0.229830,2.757960,1.128108,10.971292,7.361104


## Average of the int_reporting

In [118]:
SG_esg_int_mean = SG_esg_int.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean['year'] = SG_esg_int_mean['year'].replace([SG_esg_int_mean['year'].mean()], 'average')
SG_esg_int_mean = pd.concat([SG_esg_int, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
3,2020,334.0,127.000000,-1.143442,-0.699302,0.883334,0.387884,0.882658,10.591900,11.735342,...,-0.124728,-1.496742,-0.353300,-0.797831,-0.098530,-0.133762,-1.605140,-0.461698,-0.821496,-0.122194
4,2021,334.0,73.000000,1.629852,3.610187,0.195425,0.204391,0.194862,2.338343,0.708491,...,0.192390,2.308680,0.678829,7.260760,3.650573,0.229830,2.757960,1.128108,10.971292,7.361104
5,2022,334.0,71.000000,NaN,NaN,-0.243781,-0.230457,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,334.0,123.166667,0.216082,1.049945,0.201457,0.108938,0.309769,3.717234,3.673208,...,0.025635,0.307622,0.091540,1.754305,0.704360,0.025534,0.306407,0.090325,2.303032,1.253087


## T-test of the int_reporting

a: an array of sample observations <br>
popmean: the expected population mean (The population mean is an average of a group characteristic.) <br>

In [119]:
SG_res_dict = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, SG_esg_int.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=SG_esg_int.dropna()[SG_esg_int.columns[column]], popmean=0)
    temp_dict = {SG_esg_int.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict = {**SG_res_dict, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict[i]
                                    for i in SG_res_dict.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict

,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y2,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,t-score,0.058246,0.916284,1.537491,1.868630,1.520977,1.520977,1.357593,1.012950,1.009868,...,0.190285,0.190285,0.604447,0.993460,0.999292,0.18066,0.18066,0.355766,0.911925,0.834776
1,p-value,0.957215,0.427087,0.221775,0.158468,0.225608,0.225608,0.267688,0.385682,0.386942,...,0.861234,0.861234,0.588199,0.393715,0.391295,0.86815,0.86815,0.745562,0.429046,0.465099


In [120]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df])
SG_esg_int.tail()

,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
5,2022,334.0,71.000000,NaN,NaN,-0.243781,-0.230457,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,334.0,123.166667,0.216082,1.049945,0.201457,0.108938,0.309769,3.717234,3.673208,...,0.025635,0.307622,0.091540,1.754305,0.704360,0.025534,0.306407,0.090325,2.303032,1.253087
0,t-score,NaN,NaN,0.058246,0.916284,1.537491,1.868630,1.520977,1.520977,1.357593,...,0.190285,0.190285,0.604447,0.993460,0.999292,0.180660,0.180660,0.355766,0.911925,0.834776
1,p-value,NaN,NaN,0.957215,0.427087,0.221775,0.158468,0.225608,0.225608,0.267688,...,0.861234,0.861234,0.588199,0.393715,0.391295,0.868150,0.868150,0.745562,0.429046,0.465099


### Export the result into excel file

In [121]:
sort_companies_data_bottom20per.to_excel(bottom_path, sheet_name="main", index=False)
SG_esg_int.to_excel(reporting_path, sheet_name="main", index=False)
